In [53]:
GROQ_API_KEY = ""

In [2]:
from crewai import Agent, Task, Crew, Process
from groq import Groq
from langchain_groq import ChatGroq
import numpy as np
from crewai_tools import FileReadTool

c:\Users\USER\Documents\Egitim\.venv\Lib\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [44]:
llama3 = ChatGroq(temperature=0, model_name="llama3-70b-8192", api_key=GROQ_API_KEY)

## Deney1: Bulgulara göre Hastalık Tanısı koyan Ajan

In [5]:
%pip install PyPDF2

Note: you may need to restart the kernel to use updated packages.


In [10]:
from PyPDF2 import PdfReader

try:
    reader = PdfReader('Skin_Diseases_and_Symptoms.pdf')
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    print(text)
except Exception as e:
    print(f"Error reading PDF: {e}")

Common Skin Diseases and Their Symptoms
Acne
Description: Acne is a skin condition that occurs when hair follicles become clogged with oil and
dead skin cells.
Symptoms:
- Blackheads
- Whiteheads
- Pimples
- Redness around the skin eruptions
Eczema (Atopic Dermatitis)
Description: Eczema is a condition that makes your skin red and itchy. It is common in children but
can occur at any age.
Symptoms:
- Dry skin
- Itching, which may be severe, especially at night
- Red to brownish-gray patches, particularly on the hands, feet, ankles, wrists, neck, upper chest, eyelids, and inside the bend of the elbows and knees
- Small, raised bumps, which may leak fluid and crust over when scratched
Psoriasis
Description: Psoriasis is a skin disease that causes red, itchy scaly patches, most commonly on the
knees, elbows, trunk, and scalp.
Symptoms:
- Red patches of skin covered with thick, silvery scales
- Dry, cracked skin that may bleed or itch
- Itching, burning, or soreness- Thickened, pitted, or r

In [11]:
soru = input("What are your symptoms? ")
print(soru)

Red rash Dry, cracked skin Itching Blisters Swelling


In [24]:
# Define the agent
agent1 = Agent(
    role='Dermatologist',
    goal='Define patient\'s skin disease with given symptoms.',
    backstory="You are Prof. at Dermatology and you can easily say the skin disease with given symptoms",
    llm=llama3, 
    verbose=True,  
)

task1 = Task(
    description=f"Answer user's question {soru} with using {text} this informations.",
    expected_output="Create a txt file and save the output in it. It must only contains the disease name!",
    agent=agent1,
    output_file="result.txt"
)

In [27]:
my_crew = Crew(agents=[agent1],
               tasks=[task1], 
               process=Process.sequential,
               verbose=1,
               max_rpm=29)
crew = my_crew.kickoff()

2024-08-16 17:49:08,366 - 14472 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


 [2024-08-16 17:49:08][DEBUG]: == Working Agent: Dermatologist
 [2024-08-16 17:49:08][INFO]: == Starting Task: Answer user's question Red rash Dry, cracked skin Itching Blisters Swelling with using Common Skin Diseases and Their Symptoms
Acne
Description: Acne is a skin condition that occurs when hair follicles become clogged with oil and
dead skin cells.
Symptoms:
- Blackheads
- Whiteheads
- Pimples
- Redness around the skin eruptions
Eczema (Atopic Dermatitis)
Description: Eczema is a condition that makes your skin red and itchy. It is common in children but
can occur at any age.
Symptoms:
- Dry skin
- Itching, which may be severe, especially at night
- Red to brownish-gray patches, particularly on the hands, feet, ankles, wrists, neck, upper chest, eyelids, and inside the bend of the elbows and knees
- Small, raised bumps, which may leak fluid and crust over when scratched
Psoriasis
Description: Psoriasis is a skin disease that causes red, itchy scaly patches, most commonly on the
k

## Deney2 : Ev Önerisi Yapan Ajan

In [45]:
import pandas as pd

house_data = pd.read_csv("Housing.csv")
house_data.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [46]:
house_data.min(), house_data.max()

(price                 1750000
 area                     1650
 bedrooms                    1
 bathrooms                   1
 stories                     1
 mainroad                   no
 guestroom                  no
 basement                   no
 hotwaterheating            no
 airconditioning            no
 parking                     0
 prefarea                   no
 furnishingstatus    furnished
 dtype: object,
 price                  13300000
 area                      16200
 bedrooms                      6
 bathrooms                     4
 stories                       4
 mainroad                    yes
 guestroom                   yes
 basement                    yes
 hotwaterheating             yes
 airconditioning             yes
 parking                       3
 prefarea                    yes
 furnishingstatus    unfurnished
 dtype: object)

In [47]:
soru = input("I can find the best house for you. What are the requirements for your home? ")
print(soru)

price: 2000000, bedrooms:2+


In [51]:
# Define the agent
agent1 = Agent(
    role='Real Estate Agent',
    goal='Find appropriate house for your customer with given requirements.',
    backstory="You are an expert Real Estate Agent and you can easily find appropriate houses for your customers with given requirements.",
    llm=llama3, 
    verbose=True,  
)

task1 = Task(
    description=f"Find appropriate house with given requirements {soru} with using {house_data} this informations. Note:Information of houses is in the {house_data}! Don't exceed customer's budget and also the other requirements are must be satisfied!",
    expected_output="Create a txt file and save the output in it. It must only contains the appropriate houses! Don't duplicate the results",
    agent=agent1,
    output_file="list_of_houses.txt"
)

In [52]:
my_crew = Crew(agents=[agent1],
               tasks=[task1], 
               process=Process.sequential,
               verbose=False,
               max_rpm=29)
crew = my_crew.kickoff()

2024-08-16 18:26:08,359 - 14472 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed




> Entering new CrewAgentExecutor chain...
Thought: I now can give a great answer

Final Answer:

Here is the list of appropriate houses that meet the customer's requirements:

540   1820000  3000         2          1        1      yes        no      yes   
542   1750000  3620         2          1        1      yes        no       no   
544   1750000  3850         3          1        2      yes        no       no   

These houses have a price of 2000000 or less, 2 or more bedrooms, and meet the other specified requirements. The output has been saved to a txt file.

> Finished chain.
